In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from flask import Flask, request, jsonify
import pymysql
import numpy as np
import math
from sklearn.metrics.pairwise import cosine_similarity

app = Flask(__name__)

# Database configuration
database_name = 'db-wisata'
engine = create_engine(f'mysql+pymysql://root:@localhost/{database_name}')

# Fungsi untuk menghitung jarak Haversine
def haversine(lat1, lon1, lat2, lon2):
    r = 6371  # Radius Bumi dalam kilometer
    phi1 = math.radians(lat1)
    phi2 = math.radians(lat2)
    delta_phi = math.radians(lat2 - lat1)
    delta_lambda = math.radians(lon1 - lon2)
    a = math.sin(delta_phi / 2)**2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return r * c  # Jarak dalam kilometer

@app.route('/rekomendasi', methods=['POST'])
def rekomendasi_wisata():
    data = request.json
    if data is None:
        return jsonify({"error": "Invalid JSON"}), 400

    # Validasi data lat-long
    try:
        latitude_user = float(data['latitude'])
        longitude_user = float(data['longitude'])
    except ValueError:
        return jsonify({"error": "Invalid latitude or longitude"}), 400

    # Ambil preferensi pengguna
    jenis_wisata_user = data.get('jenis_wisata', [])
    fasilitas_wisata_user = data.get('fasilitas_wisata', [])
    
    # Tampilkan preferensi pengguna
    print("Preferensi pengguna:")
    print(f"Jenis Wisata: {jenis_wisata_user}")
    print(f"Fasilitas Wisata: {fasilitas_wisata_user}")

    # Mengambil data wisata dari database
    try:
        data_wisata = pd.read_sql('SELECT * FROM wisata', con=engine)
        print("Data wisata berhasil diambil:")
        print(data_wisata)  # Menampilkan data wisata
    except Exception as e:
        return jsonify({"error": f"Error fetching data from database: {e}"}), 500

    # One-Hot Encoding untuk jenis wisata dan fasilitas
    data_wisata = pd.get_dummies(data_wisata, columns=['jenis_wisata', 'fasilitas_wisata'], prefix=['jenis', 'fasilitas'])
    jenis_columns = [col for col in data_wisata.columns if col.startswith('jenis_')]
    fasilitas_columns = [col for col in data_wisata.columns if col.startswith('fasilitas_')]
    print("Kolom jenis dan fasilitas setelah One-Hot Encoding:")
    print(jenis_columns)
    print(fasilitas_columns)

    # Membuat vektor preferensi pengguna untuk jenis dan fasilitas
    user_jenis_vector = {col: 1 if col.split('_')[1] in jenis_wisata_user else 0 for col in jenis_columns}
    user_fasilitas_vector = {col: 1 if col.split('_')[1] in fasilitas_wisata_user else 0 for col in fasilitas_columns}
    
    # DataFrame vektor preferensi
    preference_vector_jenis = pd.DataFrame([user_jenis_vector])
    preference_vector_fasilitas = pd.DataFrame([user_fasilitas_vector])
    print("Vektor preferensi pengguna untuk jenis wisata:")
    print(preference_vector_jenis)
    print("Vektor preferensi pengguna untuk fasilitas wisata:")
    print(preference_vector_fasilitas)

    # Matriks jenis dan fasilitas pada data wisata
    wisata_matrix_jenis = data_wisata[jenis_columns].to_numpy()
    wisata_matrix_fasilitas = data_wisata[fasilitas_columns].to_numpy()

    # Hitung cosine similarity untuk jenis dan fasilitas secara terpisah
    jenis_similarity_scores = cosine_similarity(preference_vector_jenis, wisata_matrix_jenis).flatten()
    fasilitas_similarity_scores = cosine_similarity(preference_vector_fasilitas, wisata_matrix_fasilitas).flatten()
    print("Skor similarity untuk jenis wisata:")
    print(jenis_similarity_scores)
    print("Skor similarity untuk fasilitas wisata:")
    print(fasilitas_similarity_scores)

    # Perhitungan jarak Haversine dan pembobotan akhir
    recommendations = []
    for index, row in data_wisata.iterrows():
        distance = haversine(latitude_user, longitude_user, row['latitude'], row['longitude'])
        normalized_distance = 1 - (distance / (1 + distance))  # Normalisasi jarak

        # Berikan bobot pada setiap komponen
        jenis_similarity_weight = 0.5
        fasilitas_similarity_weight = 0.3
        distance_weight = 0.2

        # Hitung skor akhir dengan pembobotan
        final_score = (
            jenis_similarity_scores[index] * jenis_similarity_weight +
            fasilitas_similarity_scores[index] * fasilitas_similarity_weight +
            normalized_distance * distance_weight
        )

        # Dapatkan nilai jenis dan fasilitas wisata yang terkait dengan data wisata
        jenis_wisata = [col.split('_')[1] for col in jenis_columns if row[col] == 1]
        fasilitas_wisata = [col.split('_')[1] for col in fasilitas_columns if row[col] == 1]

        # Ambil data detail wisata dari tabel 'detail_wisata'
        try:
            detail_wisata = pd.read_sql(f"SELECT * FROM detail_wisata WHERE nama_tempat = '{row['nama']}'", con=engine)
            if not detail_wisata.empty:
                deskripsi = detail_wisata.iloc[0]['deskripsi']
                foto_url = detail_wisata.iloc[0]['foto_url']
                rating = detail_wisata.iloc[0]['rating']  # Menambahkan rating
                jumlah_ulasan = detail_wisata.iloc[0]['jumlah_ulasan'] if detail_wisata.iloc[0]['jumlah_ulasan'] is not None else 0  # Menambahkan nilai default 0
            else:
                deskripsi = "Deskripsi tidak tersedia"
                foto_url = "https://via.placeholder.com/150"
                rating = "Tidak ada rating"
                jumlah_ulasan = 0
        except Exception as e:
            deskripsi = "Deskripsi tidak tersedia"
            foto_url = "https://via.placeholder.com/150"
            rating = "Tidak ada rating"
            jumlah_ulasan = 0
            print(f"Error fetching detail wisata: {e}")

        recommendations.append({
            'nama_wisata': row['nama'],
            'final_score': final_score,
            'jenis_similarity': jenis_similarity_scores[index],
            'fasilitas_similarity': fasilitas_similarity_scores[index],
            'distance': distance,
            'normalized_distance': normalized_distance,
            'jenis_wisata': jenis_wisata,  # Menggunakan list yang ditemukan
            'fasilitas_wisata': fasilitas_wisata,  # Menggunakan list yang ditemukan
            'latitude': row['latitude'],  # Menambahkan latitude
            'longitude': row['longitude'],  # Menambahkan longitude
            'deskripsi': deskripsi,  # Menambahkan deskripsi
            'foto_url': foto_url,  # Menambahkan URL foto
            'rating': float(rating) if isinstance(rating, (int, float)) else 0.0,  # Pastikan rating tetap float
            'jumlah_ulasan': int(jumlah_ulasan) if isinstance(jumlah_ulasan, (int, float)) else 0  # Pastikan jumlah ulasan tetap integer
        })

    # Urutkan rekomendasi berdasarkan skor akhir
    recommendations = sorted(recommendations, key=lambda x: x['final_score'], reverse=True)

    # Kembalikan 10 rekomendasi teratas
    print("Rekomendasi wisata akhir:")
    print(recommendations[:10])
    return jsonify(recommendations[:10])

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False)


 * Tip: There are .env or .flaskenv files present. Do "pip install python-dotenv" to use them.


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


Preferensi pengguna:
Jenis Wisata: ['Wisata Religi']
Fasilitas Wisata: ['Tempat Parkir']
Data wisata berhasil diambil:
      id                                               nama  \
0      2                                         Ketep Pass   
1      3                              Taman Rekreasi Mendut   
2      4  AGRO WISATA KEBUN KLENGKENG DAN ISTANA KOI MAG...   
3      5                 Wisata Petik Salak Pondoh Magelang   
4      6                               Wana Wisata Sutopati   
..   ...                                                ...   
168  170                          Sendang Manis Kolokendang   
169  171                  Masjid Panembahan Santri Muntilan   
170  172                               Kolam Renang Semilir   
171  173                              Joglo kampoeng merapi   
172  174  Tingal Art Batik Borobudur ???????????????????...   

                                    jenis_wisata  \
0                                    Wisata Alam   
1                   

127.0.0.1 - - [15/Jan/2025 07:29:11] "POST /rekomendasi HTTP/1.1" 200 -


...

Rekomendasi wisata akhir:
[{'nama_wisata': 'Makam Simbah Ronggo Satoto', 'final_score': np.float64(0.8301009997641643), 'jenis_similarity': np.float64(1.0), 'fasilitas_similarity': np.float64(1.0), 'distance': 5.644297583700303, 'normalized_distance': 0.15050499882082136, 'jenis_wisata': ['Wisata Religi'], 'fasilitas_wisata': ['Tempat Parkir'], 'latitude': -7.6466544, 'longitude': 110.2400267, 'deskripsi': 'tst  tst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsi', 'foto_url': 'https://itbox.id/wp-content/uploads/2022

127.0.0.1 - - [15/Jan/2025 18:21:14] "POST /rekomendasi HTTP/1.1" 200 -


Rekomendasi wisata akhir:
[{'nama_wisata': 'Makam Simbah Ronggo Satoto', 'final_score': np.float64(0.8301009997641643), 'jenis_similarity': np.float64(1.0), 'fasilitas_similarity': np.float64(1.0), 'distance': 5.644297583700303, 'normalized_distance': 0.15050499882082136, 'jenis_wisata': ['Wisata Religi'], 'fasilitas_wisata': ['Tempat Parkir'], 'latitude': -7.6466544, 'longitude': 110.2400267, 'deskripsi': 'tst  tst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsitst dskripsi', 'foto_url': 'https://itbox.id/wp-content/uploads/2022

127.0.0.1 - - [15/Jan/2025 18:22:11] "POST /rekomendasi HTTP/1.1" 200 -


Rekomendasi wisata akhir:
[{'nama_wisata': 'Taman Dewari/Taman Bunga Matahari', 'final_score': np.float64(0.7563813869448751), 'jenis_similarity': np.float64(1.0), 'fasilitas_similarity': np.float64(0.7071067811865475), 'distance': 3.519840139991588, 'normalized_distance': 0.22124676294455436, 'jenis_wisata': ['Wisata Alam'], 'fasilitas_wisata': ['Tempat Parkir'], 'latitude': -7.6330387, 'longitude': 110.2827107, 'deskripsi': 'Deskripsi tidak tersedia', 'foto_url': 'https://via.placeholder.com/150', 'rating': 0.0, 'jumlah_ulasan': 0}, {'nama_wisata': 'TOP SELFIE GONDOSULI', 'final_score': np.float64(0.7482239359363128), 'jenis_similarity': np.float64(1.0), 'fasilitas_similarity': np.float64(0.7071067811865475), 'distance': 4.541409325711369, 'normalized_distance': 0.18045950790174248, 'jenis_wisata': ['Wisata Alam'], 'fasilitas_wisata': ['Tempat Parkir'], 'latitude': -7.5672934, 'longitude': 110.2854794, 'deskripsi': 'Deskripsi tidak tersedia', 'foto_url': 'https://via.placeholder.com/